In [7]:
#imports here
import os, csv, threading, multiprocessing, time, random, itertools, math
from ListingScraper import ListingScraper
from multiprocessing import Pool, cpu_count

In [8]:
used_words = set()

def start_scrape(urls, savePath):
    listings = ListingScraper()
    for url in urls:
        listings.fetch_data(url, savePath)

def random_words(num_words, used_words=set()):
    # load a list of English words
    with open('words.txt', 'r') as f:
        words = [line.strip() for line in f]

    if len(used_words) == len(words):
        raise Exception("All words have been used")

    # choose num_words random words from the list
    random_words = []
    for i in range(num_words):
        # choose a random word from the list
        word = random.choice(words)

        # check if the word has already been used
        if word in used_words:
            # if it has, generate a new word until we find one that hasn't been used
            while word in used_words:
                word = random.choice(words)
        else:
            # if it hasn't been used, add it to the set of used words
            used_words.add(word)

        random_words.append(word)

    return random_words

def to_url(word):
    return f"https://www.facebook.com/marketplace/106246092739961/search?query={word}"

def random_urls(url_count: int):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    urls = []

    looping = math.ceil(math.log(url_count,len(alphabet)))

    for i in range(looping):
        for word in itertools.product(alphabet, repeat=i+1):
            urls.append(to_url(''.join(word)))
            if len(urls) == url_count:
                return urls

def thread_urls(threads, per_thread: int):
    words_count = per_thread * threads
    urls = random_urls(words_count)

    # Create a list of lists to store the split URLs
    split_urls = [[] for _ in range(threads)]

    # Iterate over the URLs and assign them to the appropriate list in split_urls
    for i, url in enumerate(urls):
        split_urls[i % threads].append(url)

    return split_urls


In [9]:
basePath = '../../Desktop/Uni/Year3-Project/CSV_Files/All_Listings/Links'
url = "https://www.facebook.com/marketplace/"
threads_used = 1
words_count = 10000

paths = [basePath+f"{i}.csv" for i in range(threads_used)]

urls_lists = [random_urls(words_count) for i in range(threads_used)]

urls2 = thread_urls(threads_used,words_count)

In [10]:
threads = []
for i in range(threads_used):
    threads.append(threading.Thread(target=start_scrape, args=(urls_lists[i],paths[i])))

for thread in threads:
    thread.start()
    time.sleep(2)

Exception in thread Thread-6 (start_scrape):
Traceback (most recent call last):
  File "C:\Users\zorko\anaconda3\envs\bs4\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\zorko\anaconda3\envs\bs4\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\zorko\AppData\Local\Temp\ipykernel_27812\3256174749.py", line 6, in start_scrape
  File "C:\Users\zorko\Python\mrktplc_scraper\Scrapers.py", line 66, in fetch_data
    while self.__driver.current_url != url and attempts != 2:
  File "C:\Users\zorko\anaconda3\envs\bs4\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 530, in current_url
    return self.execute(Command.GET_CURRENT_URL)["value"]
  File "C:\Users\zorko\anaconda3\envs\bs4\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 440, in execute
    self.error_handler.check_response(response)
  File "C:\Users\zorko\anaconda3\envs\bs4\lib\site-packages\selenium\webdriver\remote\errorh